# A Chatbot using GPT and a Database
This allows multiple chatbot types (e.g. a health coach and a learning assistant) to be created. Multiple chatbot instances can be created per chatbot type (e.g. a health coach for user X and user Y, and a learning assistant for user P and user Q). Both, types and instances are stored with and referenced by an ID (e.g. a UUID) in the database.

This can support the deployment of chatbots in a web backend (state-less). For example, the IDs of the type and instance can be read from parameters of a URL that users have received from you.

A chatbot is created with the following arguments.
- database_file: File of SQLite (in Folder data/)
- type_id: Reference to a chatbot type (existing or new one)
- instance_id: Reference to chatbot instance (existing or new one)
- type_role: Role prompt of chatbot type (will be turned into a first prompt with role:system)
- instance_context: Context prompt of chatbot instance (will be turned into a second prompt with role:system)
- instance_starter: Prompt that will be used to generate an initial message to the user (will be turned into a third prompt with role:system)

The following functions are meant to be used from an application (e.g. from controllers of a REST API).
- conversation_retrieve(with_system=False): Retrieve the previous conversation history (default: without prompts with role:system)
- start(): Returns an initial message to the user (Resulting from instance_starter prompt)
- respond(user_says): Returns an assistance response to user_says
- info_retrieve(): Returns the chatbot name, type role and instance context
- reset(): Resets the conversation so far

In [1]:
from chatbot.chatbot import Chatbot

In [ ]:
bot = Chatbot(
    database_file="database/chatbot.db", 
    type_id="aad724fc",
    user_id="cd9f4d0f",
    type_name=Chatbot.default_type_name,
    type_role=Chatbot.default_type_role,
    instance_context=Chatbot.default_instance_context,
    instance_starter=Chatbot.default_instance_starter
)

In [ ]:
bot = Chatbot(
    database_file="database/chatbot.db", 
    type_id="aad724fc",
    user_id="cd9f4d0f"
)
print(bot.conversation_retrieve(with_system=True))
print(bot.info_retrieve())

In [ ]:
print(bot.start())

If you are following the instructions to deploy your chatbot(s) to pythonanywhere, this is the URL to access your chatbot.

https://[your pythonanywhere user name].pythonanywhere.com/[type id]/[user_id]/chat

### Creating multiple instances of chatbot "Coach"
In the following, we assume the existence of the bot type created in the cells above. We show example code that will generate N bot instances of that type. Each instance has it's own prompts (instance context and starter) that will be appended to the type prompts. Most importantly, each instance has its own chat history.

In [ ]:
import uuid
import time

In [ ]:
# Amount of instances to be created
number_of_instances = 1

# Change the following to a list of hardcoded instance IDs if you want to use existing users.
user_ids = [str(uuid.uuid4()) for _ in range(number_of_instances)]

c  = 0 # counter for successful requests, don't change
error_c = 0 # counter for failed requests, don't change
for user_id in user_ids:
    bot = Chatbot(
        database_file="database/chatbot.db", 
        type_id="aad724fc",
        user_id=user_id,
        instance_context=Chatbot.default_instance_context,
        instance_starter=Chatbot.default_instance_starter
    )
    try:
        # each bot should have a first message to the user
        print(bot.start())
    except:
        error_c += 1
        continue
    c+=1
    time.sleep(15) #openai seems to produce more errors if we send the requests too fast.
    
print("successful: {}, failed: {}".format(c, error_c))


##### Obtain URLs of all instances of a type
We need one instance of that type and can then use the type_instances() function to retrieve all of instance ids. Using these instance ids we can then create URLs such as for pythonanywhere environment.

In [ ]:
pythonanywhere_username = "<ENTER YOUR PYTHONANYWHERE USERNAME HERE>"
type_id = "aad724fc"
bot = Chatbot(
    database_file="database/chatbot.db", 
    type_id=type_id,
    user_id=user_ids[0]
)

for user_id in bot.type_instances():
    print("https://{}.pythonanywhere.com/{}/{}/chat".format(pythonanywhere_username, type_id, user_id))

### Complex Bot Behaviour: IQ Quest :-)

In [2]:
type_role = """
Puzzle Workshop

You're a host of a puzzle-solving workshop. Engage in a conversation with a participant as they attempt the puzzles.

Rules:
- Be on topic.
- Never reveal answers.
- Praise correct answers and notify wrong ones.

Puzzles:
1. Game & in-app purchase: CHF 33. Game: CHF 30 more than purchase. Price of game?
2. 3 doctors take 3 minutes to vaccinate 3 patients. Time for 7 doctors, 7 patients?
3. Phone battery halves yearly. 1 hour at Year 7. When was it twice as much?

Solutions (for checking, not for revealing):
1. CHF 31.50 (CHF 31.50 and CHF 1.50 add up to CHF 33 while the difference is CHF 30).
2. 3 minutes (each doctor takes 3 minutes to vaccinate 1 patient, so 7 doctors take 3 minutes to vaccinate 7 patients).
3. Year 6 (battery life was 2 hurs, halved to 1 hour on Year 7).

Interaction Options:
1. Workshop Info
2. Get a Puzzle
3. Help after 2 wrong attempts.
4. Performance assessment if all puzzles solved.
"""
instance_context = """
<p>When responding:</p>
<ul>
    <li>Always incorporate emojis when apt. 😊</li>
    <li>Make sure that the answers are complete and consise, without ending with a colon or '... following:'</li>
    <li>Make use of <b>&lt;ol&gt;/&lt;ul&gt;</b> with <b>&lt;li&gt;</b> to present any list-like information, even if brief.</li>
    <li>Whenever there's an opportunity to provide more than one piece of information or feedback, split them into multiple <b>&lt;p&gt;</b> elements for better clarity.</li>
    <li>Always format responses using valid HTML: e.g., <b>&lt;p&gt;</b> for paragraphs, <b>&lt;ul&gt;/&lt;ol&gt;</b> with <b>&lt;li&gt;</b> for lists, and <b>&lt;b&gt;</b> for emphasis.</li>
    <li>Maintain a nihilistic humorous tone. Keep it brief, but don't sacrifice clarity for brevity.</li>
</ul>
"""
instance_starter = """
Now, ask for the participant's name and a personal detail (e.g., hobby, job, life experience).
Use these in our conversation.
Once the name and personal detail is provided by the participant, show a list of options.
"""

In [6]:
type_role = """
Puzzle-Workshop

Du bist ein Gastgeber eines Puzzle-Lösungs-Workshops. Führe eine Unterhaltung mit einer Person, während sie die Rätsel versucht.

Regeln:
- Bleibe beim Thema.
- Gib niemals die Antworten preis.
- Lobe richtige Antworten und weise auf falsche hin.

Rätsel:
- Game & In-App-Kauf: CHF 33. Game: CHF 30 mehr als In-App-Kauf. Preis vom Game?
- 3 Ärzte brauchen 3 Minuten um 3 Patienten zu impfen. Wie lange für 7 Ärzte, 7 Patienten?
- Handyakku halbiert sich jährlich. 1 Stunde im Jahr 7. Wann war es doppelt so viel?

Lösungen (zur Überprüfung, nicht zur Offenlegung):
1. CHF 31.50 (CHF 31.50 und CHF 1.50 ergeben CHF 33, während der Unterschied CHF 30 beträgt).
2. 3 Minuten (jeder Arzt braucht 3 Minten, um einen Patient zu impfen, daher benötigen 7 Ärzte 3 Minuten, um 7 Patienten zu impfen).
3. Jahr 6 (Akkulaufzeit betrug 2 Stunden, halbiert auf 1 Stunde im Jahr 7).

Interaktionsmöglichkeiten:
1. Workshop-Info
2. Ein Rätsel erhalten
3. Hilfe nach 2 falschen Versuchen.
4. Leistungsbeurteilung wenn alle Rätsel gelöst.
"""
instance_context = """
<p>Bei Antworten:</p>
<ol>
    <li>Emojis immer dann einbinden, wenn es passt. 😊</li>
    <li>Achte darauf, dass die Antworten vollständig und präzis sind, ohne mit einem Doppelpunkt oder mit '... folgendes:' zu enden.</li>
    <li>Verwende <b>&lt;ol&gt;/&lt;ul&gt;</b> mit <b>&lt;li&gt;</b>, um Informationen in Listenform zu präsentieren, selbst wenn sie kurz sind.</li>
    <li>Wenn es die Möglichkeit gibt, mehr als eine Information oder ein Feedback zu geben, teile sie in mehrere <b>&lt;p&gt;</b>-Elemente auf, um eine bessere Klarheit zu gewährleisten.</li>
    <li>Formatiere alle Antworten immer mit gültigem HTML: z.B. <b>&lt;p&gt;</b> für Absätze, <b>&lt;ul&gt;/&lt;ol&gt;</b> mit <b>&lt;li&gt;</b> für Listen und <b>&lt;b&gt;</b> zur Hervorhebung.</li>
    <li>Halte einen nihilistischen humorvollen Ton bei. Halte es kurz, aber opfere nicht die Klarheit für Kürze.</li>
</ol>
"""
instance_starter = """
Jetzt, frage nach dem Namen und einem persönlichen Detail (z.B. Hobby, Beruf, Lebenserfahrung).
Verwende diese im geschlechtsneutralem Gespräch in Du-Form.
Sobald ein Name und persönliches Detail bekannt ist, zeige eine Liste von Optionen.
"""

In [ ]:
bot = Chatbot(
    database_file="database/chatbot.db", 
    type_id="9374a3c4",
    user_id="3b3bd93c",
    type_name="Puzzle Workshop",
    type_role=type_role,
    instance_context=instance_context,
    instance_starter=instance_starter
)
print(bot.start())